In [1]:
import cv2, os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import pathlib

2024-06-24 21:34:20.022040: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 21:34:20.986204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from tensorflow.keras.models import Sequential

In [3]:
path0 = "/home/rick/Ri/SecondYear/2ndSemester/AI-Lab/project_private/train"
path1 = "/home/rick/Ri/SecondYear/2ndSemester/AI-Lab/project_private/test"
data_dir0 = pathlib.Path(path0).with_suffix('')
data_dir1 = pathlib.Path(path1).with_suffix('')

In [4]:
image_count = len(list(data_dir0.glob('*/*.jpg')))
print(image_count)

6366


In [5]:
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir0,
  validation_split=0.2,
  seed = 123,
  subset="training",
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6366 files belonging to 5 classes.
Using 5093 files for training.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir0,
  validation_split=.2,
  seed = 123,
  subset="validation",
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6366 files belonging to 5 classes.
Using 1273 files for validation.


In [7]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir1, 
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 707 files belonging to 5 classes.


In [9]:
data_augmentation = tf.keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

/home/rick/Ri/SecondYear/2ndSemester/AI-Lab/AI-Lab_env/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
class_names = train_ds.class_names
print(class_names)

['FoldingKnife', 'Multi-tool Knife', 'Scissor', 'Straight Knife', 'Utility Knife']


In [11]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 180, 180, 3)
(32,)


In [12]:
num_classes = len(class_names)
print(num_classes)

5


In [24]:
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [14]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [30]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [131]:
model.compile(optimizer=tf.keras.optimizers.Adamax(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputs (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.save("sequential_final.keras")

In [112]:
model = tf.keras.models.load_model("current1.keras")

In [ ]:
epochs = 50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 79s 484ms/step - accuracy: 0.1162 - loss: 8.0220 - val_accuracy: 0.1037 - val_loss: 8.5942
Epoch 2/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 77s 483ms/step - accuracy: 0.1130 - loss: 8.0636 - val_accuracy: 0.1037 - val_loss: 8.5942
Epoch 3/50
 29/160 ━━━━━━━━━━━━━━━━━━━━ 1:00 461ms/step - accuracy: 0.1310 - loss: 7.7779

In [ ]:
def print_accuracy(history) -> None:
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs_range = range(epochs)
    
    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    return

In [108]:
print_accuracy(model.history)

AttributeError: 'Sequential' object has no attribute 'history'

In [68]:
"""import PIL
images = list(data_dir1.glob('*/*'))
print(str(images[0]))
img_array = tf.keras.utils.img_to_array(PIL.Image.open(str(images[0])))
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)"""

'import PIL\nimages = list(data_dir1.glob(\'*/*\'))\nprint(str(images[0]))\nimg_array = tf.keras.utils.img_to_array(PIL.Image.open(str(images[0])))\npredictions = model.predict(img_array)\nscore = tf.nn.softmax(predictions[0])\n\nprint(\n    "This image most likely belongs to {} with a {:.2f} percent confidence."\n    .format(class_names[np.argmax(score)], 100 * np.max(score))\n)'

In [110]:
results = model.evaluate(test_ds)
print(f'Test Loss: {results[0]}')
print(f'Test Accuracy: {results[1]}')

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_3" is incompatible with the layer: expected axis -1 of input shape to have value 65536, but received input with shape (None, 30976)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 180, 180, 3), dtype=float32)
  • training=False
  • mask=None

In [70]:
predictions = model.predict(test_ds)

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step


In [71]:
predicted_class_indices = np.argmax(predictions, axis=1)
print(predicted_class_indices)

[2 2 2 0 2 2 0 0 2 2 1 2 2 1 2 1 0 2 2 2 4 2 0 2 2 2 2 2 0 2 1 2 2 2 2 2 2
 2 2 2 4 2 2 0 2 0 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 1 2
 2 2 4 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 1 2 2 2 1 0 2 2 2 0 2 2 1
 2 2 2 2 2 2 2 0 2 2 2 0 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0
 2 2 2 2 0 1 2 0 2 2 2 2 2 0 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 0 2 0 2 0 2 2 2
 2 0 2 2 2 4 2 2 1 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 0 2 1 2 2
 2 2 1 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 0 2 0 0 2 2 1 2 0 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 1 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 0 2 1
 1 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 1 2 0 2 2 2 2 2 2 2 2 2 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 2 2 0 2 2 2 2 2 0 2 2 0 2 2 2 1 0 0 2 2 2 2 2 0 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 0 2 1 2 2 2 2 2 2 2 0 1 2 2 2
 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 1 2 2 2 2 2 2 1 